# Costruzioni dataset

In [8]:
import os
import sentencepiece as spm
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom

## Dataset per RDF2Text

### Train e dev set

In [14]:
dataset_types = ["train", "dev"] 
triple_numbers = ["1", "2", "3", "4", "5", "6", "7"]

i = 0
for dataset_type in dataset_types:
    dataset = []

    for triple_number in triple_numbers:
        path = "..\\..\\webnlg\\it-PE\\" + dataset_type + "\\" + triple_number + "triples"
        file_names = []
        for file_name in os.listdir(path):
            if os.path.isfile(os.path.join(path, file_name)):
                url = os.path.join(path, file_name)
                tree = ET.parse(url)
                root = tree.getroot()

                for entry in root.iter('entry'):
                    mts = []

                    eid = entry.get('eid')
                    category = entry.get('category')
                    shape = entry.get('shape')
                    shape_type = entry.get('shape_type')
                    size = entry.get('size')
                    
                    for modifiedtripleset in entry.iter('modifiedtripleset'):
                        mtriples = ""

                        # con ordinamento alfabetico
                        triples = []
                        for mtriple in modifiedtripleset.iter('mtriple'):  
                            triple = mtriple.text.split(" | ")
                            triple = (triple[0], triple[1], triple[2])
                            triples.append(triple)

                        triples.sort(key=lambda x: x[1])
                        for triple in triples:
                            mtriples += triple[0] + " " + triple[1] + " " + triple[2] + " "

                        # if last character is space remove it
                        if mtriples[-1] == " ":
                            mtriples = mtriples[:-1]
                            mts.append(mtriples)


                        mts.append(mtriples)

                    lexs = []
                    for lex in entry.iter('lex'):
                        if lex.get('lang') == "it-PE":
                            lexs.append(lex.text)

                    for lex in lexs:
                        i += 1
                        dataset.append([size, file_name, eid, category, shape, shape_type, mts[0], lex])

    df_dataset = pd.DataFrame(dataset)
    df_dataset.columns = ["num_triples", "file_name", "eid", "category", "shape", "shape_type", "data_unit", "sentence"]
    df_dataset.to_csv("datasets\\it-PE\\" + dataset_type + ".csv", index=False)

print(i)

39890


### Test set

In [2]:
dataset = []

path = "..\\..\\webnlg\\it-PE\\test"
file_name = "rdf-to-text-generation-test-data-with-refs-en.xml"

if os.path.isfile(os.path.join(path, file_name)):
    url = os.path.join(path, file_name)
    tree = ET.parse(url)
    root = tree.getroot()

    for entry in root.iter('entry'):
        mts = []
        
        eid = entry.get('eid')
        category = entry.get('category')
        shape = entry.get('shape')
        shape_type = entry.get('shape_type')
        size = entry.get('size')

        for modifiedtripleset in entry.iter('modifiedtripleset'):
            mtriples = ""

            # Con ordinamento alfabetico
            triples = []
            for mtriple in modifiedtripleset.iter('mtriple'):  
                triple = mtriple.text.split(" | ")
                triple = (triple[0], triple[1], triple[2])
                triples.append(triple)

            triples.sort(key=lambda x: x[1])
            for triple in triples:
                mtriples += triple[0] + " " + triple[1] + " " + triple[2] + " "
                        
            # Rimuovi l'ultimo spazio, se presente
            mtriples = mtriples.rstrip()
            mts.append(mtriples)

        # Filtra solo le frasi con lingua 'it-PE'
        lexs = [lex.text for lex in entry.iter('lex') if lex.get('lang') == 'it-PE']

        # Assicurati che mts e lexs abbiano valori predefiniti
        mts = mts if mts else [""]
        lexs = lexs if lexs else [""]

        dataset.append([size, file_name, eid, category, shape, shape_type, mts[0], lexs])

# Converti direttamente in un DataFrame
df_dataset = pd.DataFrame(dataset, columns=["num_triples", "file_name", "eid", "category", "shape", "shape_type", "data_unit", "sentence"])

# Salva il DataFrame in un file CSV
df_dataset.to_csv("datasets\\it-PE\\test.csv", index=False)